In [11]:
from __future__ import print_function
import scipy.io as sio
from matplotlib import pyplot as plt
import numpy as np


# configuration
BATCH_SIZE = 35
DATASET_PATH = "./data/train_32x32.mat"

class DataLoader:
  def __init__(self, data_path, batch_size):
    data = sio.loadmat(data_path)
    self.images = data['X']
    self.labels = data['y']

    num_samples = self.labels.shape[0]
    num_batches = num_samples // batch_size
    # last_batch_size = num_samples - num_batches * batch_size
    split_conf = [batch_size*(i+1) for i in range(num_batches)]
    split_conf.append(num_samples)

    self.image_batches = np.split(np.transpose(self.images, (3, 0, 1, 2)), split_conf, axis=0)
    self.label_batches = np.split(self.labels, split_conf, axis=0)

  def iter(self):
    for image, label in zip(self.image_batches, self.label_batches):
      yield image, label


if __name__ == "__main__":

  dataloader = DataLoader(DATASET_PATH, BATCH_SIZE)

  batch_count = 0
  fig = plt.figure()
  for image_batch, label_batch in dataloader.iter():
    if batch_count > 0:
      break
    for batch_i in range(BATCH_SIZE):
      sub_plot = fig.add_subplot(BATCH_SIZE/10, 10, batch_i+1)
      plt.imshow(image_batch[batch_i])
    batch_count += 1

  plt.show()


ValueError: num must be 1 <= num <= 30, not 31